# Двухуровневые модели рекомендаций


**Основное**  
Данны данные о покупках 2500 пользователей в течении 95 недель так же есть характеристики пользователя и характеристики продукта.
Задача разработать модель с
- Целевая метрика precision@5 > 0.22

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit import approximate_als
from implicit.nearest_neighbours import ItemItemRecommender,  CosineRecommender

# Модель второго уровня
from catboost import CatBoostClassifier
from utilis import prefilter_items, prefilter_items_v2
from recomenders5 import MainRecommender
from metrics import recall_at_k_mean, recall_at_k, precision_at_k, map_k_mean, NDCG_mean, ap_k
from data_prepare import reduce_mem_usage, nan_replacer, feature_generator,user_feature_prepare, item_features_prepare
from recomender9 import MainRecommender
import warnings
warnings.filterwarnings("ignore")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data = pd.read_csv('retail_train.csv')
#data = prefilter_items(data)

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

#column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [4]:
data = prefilter_items(data, take_n=20000)

In [5]:
max_week = data['week_no'].max()
condition_train = (data['week_no'] < max_week - 6)
condition_valid = data['week_no'] >= max_week - 3
condition_test = ((data['week_no'] >= max_week - 6) & (data['week_no'] < max_week - 3))
                   
data_train_L1 = data[condition_train]
data_test_L1 = data[condition_test]

data_train_L2 = data_test_L1.copy()
data_valid_L2 = data[condition_valid]

In [6]:
data_l1_l2_val = pd.concat([data_valid_L2, data_test_L1], ignore_index = True)

In [7]:
result = data_test_L1.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[829323, 999999, 836423, 851515, 875240, 87737..."
1,2,"[999999, 821083, 828106, 830960, 833025, 83813..."


In [8]:
result = result.loc[result['user_id']<=2499]

# ItemItem

In [9]:
from recomender9 import MainRecommender

In [10]:
%%time
rec = MainRecommender(data_train_L1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/19990 [00:00<?, ?it/s]

  0%|          | 0/19990 [00:00<?, ?it/s]

CPU times: total: 1min 13s
Wall time: 29.1 s


In [11]:
%%time
item_200 = np.array([])
for ids in result['user_id']:
    item_200 = np.append(item_200, rec.get_item_recommendations(ids, N=200))
    item_200 = item_200.reshape(-1, 200)

CPU times: total: 2.66 s
Wall time: 2.7 s


In [12]:
result['item'] = item_200.tolist()

### Полнота на 1-ом уровне на тесте

In [13]:
result.apply(lambda row: recall_at_k(row['item'], row['actual'], k=200), axis=1).mean()

0.18206286060866114

# Генерация признаков

In [14]:
user_feature = user_feature_prepare(user_features)

In [15]:
item_features = item_features_prepare(item_features)

In [16]:
user_item_features = feature_generator(data_train_L1,user_feature, item_features)

Добавлены следующие признаки:
время покупки -hour
день недели совершения транзакции-median_weekday
кол-во транзакций покупок пользователем-n_transactions
кол-во уникальных покупок пользователем-n_items
средний чек пользвателя - mean_check
средний чек на размер семьи-mean_ckeck_per_household_size
популярность товара-popularity
любимые товары пользователя-purchase_2,purchase_3,purchase_4


In [17]:
user_item_features.head(5)

,user_id,item_id,median_sales_hour,median_weekday,n_items,purchase_2,purchase_3,purchase_4,mean_check,n_transactions,...,household_size_desc,kid_category_desc,mean_ckeck_per_household_size,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,commodity_category
0,1,820165,12.0,1.5,482,856942.0,1082185.0,995242.0,50.174167,1449,...,2.0,0,25.087083,2.0,PRODUCE,1.0,CITRUS,ORANGES NAVELS ALL,,166.0
1,1,821815,16.0,3.0,482,856942.0,1082185.0,995242.0,50.174167,1449,...,2.0,0,25.087083,131.0,GROCERY,1.0,DRY SAUCES/GRAVY,GRAVY CAN/GLASS,12 OZ,132.0
2,1,823721,14.0,4.0,482,856942.0,1082185.0,995242.0,50.174167,1449,...,2.0,0,25.087083,317.0,GROCERY,1.0,CHEESE,GRATED CHEESE,8 OZ,14.0
3,1,823990,15.0,6.0,482,856942.0,1082185.0,995242.0,50.174167,1449,...,2.0,0,25.087083,2929.0,MEAT,1.0,BEEF,CHOICE BEEF,,13.0
4,1,825123,15.0,4.0,482,856942.0,1082185.0,995242.0,50.174167,1449,...,2.0,0,25.087083,1179.0,GROCERY,1.0,SALD DRSNG/SNDWCH SPRD,SEMI-SOLID SALAD DRESSING MAY,30 OZ,34.0


# Подготовка к уровню 2

### Слияние USER и IITEM + добавление flag 0 если не покупал 1 если купил

In [18]:
users_lvl_2 = pd.DataFrame(data_train_L2['user_id'].unique())
users_lvl_2.columns = ['user_id']
users_lvl_2 = users_lvl_2.loc[users_lvl_2['user_id']<=2499] # конец не попал в обучение поэтому и предсказать не сможем для юсеров выше
# Пока только warm start
train_users = data_train_L1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)] #выбрали только тех пользователей что были в 1й обучающей выборке
#Обучение 1 го уровня и создание кандидатов
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: rec.get_item_recommendations(x, N=200))

In [19]:
users_lvl_2.head(3)

,user_id,candidates
0,84,"[1082185, 981760, 995242, 1098066, 826249, 840..."
1,1753,"[1082185, 981760, 1098066, 826249, 995242, 840..."
2,2120,"[1082185, 981760, 1127831, 995242, 840361, 109..."


In [20]:
# склеивание каждого el из candidates с user_id
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'
users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1 #пока метка у всех 1
users_lvl_2.head(3)

,user_id,item_id,flag
0,84,1082185,1
0,84,981760,1
0,84,995242,1


In [21]:
targets_lvl_2 = data_train_L2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 
targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)
targets_lvl_2.head(3)

,user_id,item_id,target
0,84,1082185,0.0
1,84,981760,0.0
2,84,995242,0.0


In [22]:
targets_lvl_2.shape

(380334, 3)

# Добавление признаков на уровень 2

In [23]:
targets_lvl_2 = targets_lvl_2.merge(user_item_features, on=['user_id','item_id'], how='left')
targets_lvl_2.head(4)

,user_id,item_id,target,median_sales_hour,median_weekday,n_items,purchase_2,purchase_3,purchase_4,mean_check,...,household_size_desc,kid_category_desc,mean_ckeck_per_household_size,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,commodity_category
0,84,1082185,0.0,1.0,4.0,241.0,903529.0,1080414.0,920025.0,23.707907,...,NaN,NaN,NaN,2.0,PRODUCE,1.0,TROPICAL FRUIT,BANANAS,40 LB,216.0
1,84,981760,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84,995242,0.0,7.5,3.5,241.0,903529.0,1080414.0,920025.0,23.707907,...,NaN,NaN,NaN,69.0,GROCERY,0.0,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,,62.0
3,84,1098066,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
targets_lvl_2.shape

(380334, 27)

## Обработка признаков товаров

In [25]:
targets_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380334 entries, 0 to 380333
Data columns (total 27 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   user_id                        380334 non-null  int64   
 1   item_id                        380334 non-null  int64   
 2   target                         380334 non-null  float64 
 3   median_sales_hour              108037 non-null  float64 
 4   median_weekday                 108037 non-null  float64 
 5   n_items                        108037 non-null  float64 
 6   purchase_2                     108036 non-null  float64 
 7   purchase_3                     108036 non-null  float64 
 8   purchase_4                     108036 non-null  float64 
 9   mean_check                     108037 non-null  float64 
 10  n_transactions                 108037 non-null  float64 
 11  popularity                     108037 non-null  float64 
 12  age_desc        

In [26]:
targets_lvl_2 = nan_replacer(targets_lvl_2)

In [27]:
targets_lvl_2 = reduce_mem_usage(targets_lvl_2)

Memory usage of dataframe is 73.64 MB
Memory usage after optimization is: 33.75 MB
Decreased by 54.2%


In [28]:
targets_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380334 entries, 0 to 380333
Data columns (total 27 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   user_id                        380334 non-null  int16   
 1   item_id                        380334 non-null  int32   
 2   target                         380334 non-null  float32 
 3   median_sales_hour              380334 non-null  float32 
 4   median_weekday                 380334 non-null  float32 
 5   n_items                        380334 non-null  float32 
 6   purchase_2                     380334 non-null  float32 
 7   purchase_3                     380334 non-null  float32 
 8   purchase_4                     380334 non-null  float32 
 9   mean_check                     380334 non-null  float32 
 10  n_transactions                 380334 non-null  float32 
 11  popularity                     380334 non-null  float32 
 12  age_desc        

## Разделение на трейн и предикт

In [29]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [30]:
X_train.shape

(380334, 26)

In [31]:
X_train = X_train.loc[X_train['item_id']!=999999]

In [32]:
X_train.shape

(380334, 26)

## Предобработка обучающего множества

In [33]:
choosen_feature = ['item_id',
                'popularity',
                'n_transactions',
                'purchase_4',
                'median_sales_hour',
                'median_weekday',
                'user_id',
                'n_items',
                'mean_check',
                'commodity_category',
                'purchase_2',
                'purchase_3',
                'sub_commodity_desc',
                'manufacturer']

In [34]:
X_train = X_train[choosen_feature]

In [35]:
categorial_columns = [col for col in X_train.columns if X_train[col].dtype =='category']

In [36]:
categorial_columns

['sub_commodity_desc']

## Обучение

In [37]:
%%time
cat = CatBoostClassifier(max_depth=7, n_estimators=1200, cat_features=categorial_columns, random_state=14, silent=True)
cat.fit(X_train, y_train, cat_features=categorial_columns)

CPU times: total: 26min
Wall time: 5min 14s


In [38]:
train_preds = cat.predict(X_train)
train_preds_proba = cat.predict_proba(X_train)

In [39]:
train_preds = train_preds.astype(bool)

In [40]:
rec_items = X_train[train_preds].groupby(by=['user_id'])['item_id'].unique().reset_index()
rec_items.columns = ['user_id', 'model_preds']

In [41]:
data_valid_L2 = data_valid_L2.loc[data_valid_L2['item_id']!=999999]

In [42]:
result_l2 = data_valid_L2.groupby('user_id')['item_id'].unique().reset_index()
result_l2.columns = ['user_id', 'actual']

In [43]:
result_l2 = result_l2.merge(rec_items,
                                  on='user_id',
                                  how='inner')

In [44]:
result_l2.head(4)

,user_id,actual,model_preds
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 995242, 840361, 1005186]"
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 1024306]"
2,7,"[840386, 889774, 898068, 909714, 929067, 95347...",[1082185]
3,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 1029743]"


### Точность на Тренировочной выборке

In [45]:
result_l2.apply(lambda row: precision_at_k(row['model_preds'], row['actual']), axis=1).mean()

0.5940285204991088

При сокращнение количества признаков до 14 метрика точности на тренировочном множестве упала на 2%

In [46]:
importance = []
for f, n in zip(cat.feature_importances_, cat.feature_names_):
    importance.append((f,n))
importance.sort(reverse=True)

In [47]:
importance[:14]

[(17.257260055243325, 'item_id'),
 (9.397744208501551, 'popularity'),
 (7.793479006839854, 'n_transactions'),
 (7.279560280928382, 'purchase_4'),
 (6.748438559081776, 'n_items'),
 (6.630117467974545, 'user_id'),
 (6.465186908046023, 'commodity_category'),
 (6.104005675576925, 'mean_check'),
 (6.0175833502702165, 'purchase_2'),
 (5.898214049204149, 'median_sales_hour'),
 (5.4809661968026315, 'median_weekday'),
 (5.425869432915209, 'sub_commodity_desc'),
 (5.33474120930678, 'purchase_3'),
 (4.166833599308624, 'manufacturer')]

## Проверка на тесте

In [48]:
X_test_2 = data_valid_L2.merge(user_item_features, on=['user_id','item_id'], how='left')

In [49]:
X_test_2.shape

(104913, 36)

In [50]:
X_test_2 = nan_replacer(X_test_2)

In [51]:
X_test_2 = reduce_mem_usage(X_test_2)

Memory usage of dataframe is 27.53 MB
Memory usage after optimization is: 12.80 MB
Decreased by 53.5%


In [52]:
X_test_2=X_test_2[choosen_feature]

In [53]:
cat_proba = cat.predict_proba(X_test_2)[:, 1]
cat_predict = cat.predict(X_test_2)

In [54]:
cat_predict = cat_predict.astype(bool)

In [55]:
rec_items_test = X_test_2[cat_predict].groupby(by=['user_id'])['item_id'].unique().reset_index()
rec_items_test.columns = ['user_id', 'model_preds_test']

In [56]:
result_l2 = result_l2.merge(rec_items_test,
                                  on='user_id',
                                  how='inner')

In [57]:
result_l2.head(4)

,user_id,actual,model_preds,model_preds_test
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 995242, 840361, 1005186]","[995242, 1005186, 1082185]"
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 1024306]",[1024306]
2,7,"[840386, 889774, 898068, 909714, 929067, 95347...",[1082185],"[1082185, 1126899]"
3,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 1029743]","[1082185, 1029743]"


## Точность на тесте 

In [58]:
result_l2.apply(lambda row: precision_at_k(row['model_preds_test'], row['actual']), axis=1).mean()

1.0

In [59]:
result_2_test = X_test_2[['user_id', 'item_id']]
result_2_test['predictions'] = cat_proba

result_2_test = result_2_test.groupby(['user_id', 'item_id'])['predictions'].median().reset_index()
result_2_test = result_2_test.sort_values(['predictions'], ascending=False).groupby(['user_id']).head(5)
result_2_test = result_2_test.groupby('user_id')['item_id'].unique().reset_index()

In [60]:
result_2_test.columns = ['user_id', 'model_test_5']

In [61]:
result_l2 = result_l2.merge(result_2_test, on ='user_id', how='inner' )

In [62]:
result_l2.apply(lambda row: precision_at_k(row['model_test_5'], row['actual']), axis=1).mean()

1.0

In [63]:
NDCG_mean(result_l2['model_test_5'], result_l2['actual'])

0.9965340625151786

# Выводы
Модель показала идеальную точность отчасти из за наличия таких признаков как любимые товары и популярность товара.
Логично что чем больше раз пользователь покупал товар тем больше вероятность что пользователь вновь его купит.
И соответсвенно чем популярней товар тем больше вероятность что пользователи его купят.
Ранжирующая метрика NDCG так же близка к идеальной.
Данная модель идеально предсказывает предпочтения пользователя однако не предлагает пользователю ничего нового или схожего с тем что он брал но не то что он брал.
На AB тесте модель тоже скорей всего покажет хороший результат с любой другой моделью (по предсказанию).
Однако дополнительной прибыли модель врятли принесет поскольку не рекомендует ничего нового.
Для создания более разнообразных предложение для пользователя можно использовать bm24 tfidf взвешивание а так же уменьшить число любимых товаров.

